## 1. инпуты

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.exc import IntegrityError

## 2. ознакомление

In [4]:
df = pd.read_csv('hotel_bookings.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [8]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


## 3. работа с пропусками и дублями

In [11]:
#колво пропусков
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])

company     112593
agent        16340
country        488
children         4
dtype: int64


In [13]:
#процент пропусков
missing_summary = df.isna().mean().mul(100).round(5).to_frame('Missing_Percent')
missing_summary = missing_summary[missing_summary.Missing_Percent > 0]
missing_summary['Missing_Count'] = df.isna().sum()
missing_summary = missing_summary.sort_values('Missing_Percent', ascending=False)
missing_summary

,Missing_Percent,Missing_Count
company,94.30689,112593
agent,13.68624,16340
country,0.40874,488
children,0.00335,4


In [15]:
# записи без гостей
no_guests = df[(df['adults'] + df['children'] + df['babies']) == 0]
no_guests.shape[0]

180

In [17]:
#удаляем записи без гостей
df = df[(df['adults'] + df['children'] + df['babies']) > 0]

In [19]:
#проверка пропусков
missing1 = df.isnull().sum().sort_values(ascending=False)
print(missing1[missing1 > 0])

company    112438
agent       16278
country       478
dtype: int64


In [21]:
#пропуски стран заменяем, чтобы не терять остальные данные
df['country'] = df['country'].fillna('Unknown')

In [23]:
#проверка пропусков
missing2 = df.isnull().sum().sort_values(ascending=False)
print(missing2[missing2 > 0])

company    112438
agent       16278
dtype: int64


In [25]:
#заменяем пропуски в агентах на 0 и перевеодим в инт
missing_agent = df['agent'].isna().sum()
missing_agent
df['agent'] = df['agent'].fillna(0).astype(int)

In [27]:
#компании удаляем, процент пропуска слишком высок, набор данных в колонке не играет критичной ценности
df = df.drop(columns=['company'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119206 entries, 0 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119206 non-null  object 
 1   is_canceled                     119206 non-null  int64  
 2   lead_time                       119206 non-null  int64  
 3   arrival_date_year               119206 non-null  int64  
 4   arrival_date_month              119206 non-null  object 
 5   arrival_date_week_number        119206 non-null  int64  
 6   arrival_date_day_of_month       119206 non-null  int64  
 7   stays_in_weekend_nights         119206 non-null  int64  
 8   stays_in_week_nights            119206 non-null  int64  
 9   adults                          119206 non-null  int64  
 10  children                        119206 non-null  float64
 11  babies                          119206 non-null  int64  
 12  meal                 

In [31]:
df['children'] = df['children'].astype(int)

In [33]:
#заменяем undefined на sc (из описания известно, что это одно и то же)
undefined_count = (df['meal'] == 'Undefined').sum()
undefined_count

1169

In [35]:
df['meal'] = df['meal'].replace('Undefined', 'SC')

In [37]:
undefined_market_segment = (df['market_segment'] == 'Undefined').sum()
undefined_distribution_channel = (df['distribution_channel'] == 'Undefined').sum()

undefined_market_segment, undefined_distribution_channel

(0, 1)

In [39]:
#дубли
duplicates_count = df.duplicated().sum()
duplicates_count

31987

In [41]:
#удаляем дубли
df = df.drop_duplicates()

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87219 entries, 0 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87219 non-null  object 
 1   is_canceled                     87219 non-null  int64  
 2   lead_time                       87219 non-null  int64  
 3   arrival_date_year               87219 non-null  int64  
 4   arrival_date_month              87219 non-null  object 
 5   arrival_date_week_number        87219 non-null  int64  
 6   arrival_date_day_of_month       87219 non-null  int64  
 7   stays_in_weekend_nights         87219 non-null  int64  
 8   stays_in_week_nights            87219 non-null  int64  
 9   adults                          87219 non-null  int64  
 10  children                        87219 non-null  int32  
 11  babies                          87219 non-null  int64  
 12  meal                            8721

## 4. работа с типами данных

### 4.1. перевод категорий в формат даты для полей с датой

In [47]:
df['arrival_date_month'].unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [49]:
df['arrival_date_month'].isna().sum()

0

In [51]:
month_str_to_int = {
    'January': 1, 'February': 2, 'March': 3,
    'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9,
    'October': 10, 'November': 11, 'December': 12
}
print(month_str_to_int)

{'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}


In [53]:
df['arrival_date_month_int'] = df['arrival_date_month'].map(month_str_to_int)

df['arrival_date'] = pd.to_datetime({
    'year': df['arrival_date_year'],
    'month': df['arrival_date_month_int'],
    'day': df['arrival_date_day_of_month']
})

In [55]:
df = df.drop(columns=['arrival_date_week_number'])

In [57]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87219 entries, 0 to 119389
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           87219 non-null  object        
 1   is_canceled                     87219 non-null  int64         
 2   lead_time                       87219 non-null  int64         
 3   stays_in_weekend_nights         87219 non-null  int64         
 4   stays_in_week_nights            87219 non-null  int64         
 5   adults                          87219 non-null  int64         
 6   children                        87219 non-null  int32         
 7   babies                          87219 non-null  int64         
 8   meal                            87219 non-null  object        
 9   country                         87219 non-null  object        
 10  market_segment                  87219 non-null  object        
 11  distri

In [61]:
df = df.drop(columns=[
    'arrival_date_year',
    'arrival_date_month',
    'arrival_date_day_of_month',
    'arrival_date_month_int'
])

## 5. создание структуры

### 5.1. подключение к БД и создание таблиц

In [65]:
USER = "postgres"
PASSWORD = "123"
HOST = "localhost"
PORT = 5432
DB_NAME = "final_project"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")


In [83]:
create_countries_sql = """
CREATE TABLE IF NOT EXISTS countries (
    country_id SERIAL PRIMARY KEY,
    country_code TEXT UNIQUE NOT NULL,
    country_name TEXT
);
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_countries_sql))
        print('Таблица countries создана.')
except IntegrityError as e:
    print('countries: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('countries: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('countries: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('countries: неизвестная ошибка при загрузке:', e)

Таблица countries создана.


In [85]:
create_booking_sql = """
CREATE TABLE IF NOT EXISTS booking (
    booking_id SERIAL PRIMARY KEY,
    arrival_date DATE NOT NULL,
    reservation_status TEXT NOT NULL,
    reservation_status_date DATE NOT NULL,
    lead_time INT NOT NULL,
    days_in_waiting_list INT NOT NULL,
    stays_in_weekend_nights INT NOT NULL,
    stays_in_week_nights INT NOT NULL,
    assigned_room_type TEXT NOT NULL,
    deposit_type TEXT NOT NULL,
    hotel TEXT NOT NULL
);
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_booking_sql))
        print('Таблица booking создана.')
except IntegrityError as e:
    print('booking: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('booking: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('booking: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('booking: неизвестная ошибка при загрузке:', e)

Таблица booking создана.


In [87]:
create_client_behavior_sql = """
CREATE TABLE IF NOT EXISTS client_behavior (
    booking_id INT PRIMARY KEY REFERENCES booking(booking_id),
    booking_changes INT NOT NULL,
    is_canceled BOOLEAN NOT NULL,
    is_repeated_guest BOOLEAN NOT NULL,
    previous_cancellations INT NOT NULL,
    previous_bookings_not_canceled INT NOT NULL
);
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_client_behavior_sql))
        print('Таблица client_behavior создана.')
except IntegrityError as e:
    print('client_behavior: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('client_behavior: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('client_behavior: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('client_behavior: неизвестная ошибка при загрузке:', e)

Таблица client_behavior создана.


In [89]:
create_client_request_sql = """
CREATE TABLE IF NOT EXISTS client_request (
    booking_id INT PRIMARY KEY REFERENCES booking(booking_id),
    total_of_special_requests INT NOT NULL,
    required_car_parking_spaces INT NOT NULL,
    reserved_room_type TEXT NOT NULL,
    adults INT NOT NULL,
    children INT NOT NULL,
    babies INT NOT NULL,
    meal TEXT NOT NULL
);
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_client_request_sql))
        print('Таблица client_request создана.')
except IntegrityError as e:
    print('client_request: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('client_request: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('client_request: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('client_request: неизвестная ошибка при загрузке:', e)

Таблица client_request создана.


In [91]:
create_marketing_info_sql = """
CREATE TABLE IF NOT EXISTS marketing_info (
    booking_id INT PRIMARY KEY REFERENCES booking(booking_id),
    country_id INT REFERENCES countries(country_id),
    market_segment TEXT NOT NULL,
    distribution_channel TEXT NOT NULL,
    agent INT NOT NULL,
    customer_type TEXT NOT NULL,
    adr NUMERIC NOT NULL
);
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_marketing_info_sql))
        print('Таблица marketing_info создана.')
except IntegrityError as e:
    print('marketing_info: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('marketing_info: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('marketing_info: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('marketing_info: неизвестная ошибка при загрузке:', e)

Таблица marketing_info создана.


### 5.2. заполнение таблиц

#### 5.2.1. страны

In [93]:
# Шаг 1: Полный словарь стран
ru_code_name = {
    'ABW': 'Аруба', 'AGO': 'Ангола', 'AIA': 'Ангилья', 'ALB': 'Албания', 'AND': 'Андорра',
    'ARE': 'ОАЭ', 'ARG': 'Аргентина', 'ARM': 'Армения', 'ASM': 'Американское Самоа',
    'ATA': 'Антарктида', 'ATF': 'Французские Южные территории', 'AUS': 'Австралия',
    'AUT': 'Австрия', 'AZE': 'Азербайджан', 'BDI': 'Бурунди', 'BEL': 'Бельгия',
    'BEN': 'Бенин', 'BFA': 'Буркина-Фасо', 'BGD': 'Бангладеш', 'BGR': 'Болгария',
    'BHR': 'Бахрейн', 'BHS': 'Багамские Острова', 'BIH': 'Босния и Герцеговина',
    'BLR': 'Беларусь', 'BOL': 'Боливия', 'BRA': 'Бразилия', 'BRB': 'Барбадос',
    'BWA': 'Ботсвана', 'CAF': 'ЦАР', 'CHE': 'Швейцария', 'CHL': 'Чили', 'CHN': 'Китай',
    'CIV': 'Кот-д’Ивуар', 'CMR': 'Камерун', 'CN': 'Китай', 'COL': 'Колумбия',
    'COM': 'Коморы', 'CPV': 'Кабо-Верде', 'CRI': 'Коста-Рика', 'CUB': 'Куба',
    'CYM': 'Каймановы острова', 'CYP': 'Кипр', 'CZE': 'Чехия', 'DEU': 'Германия',
    'DJI': 'Джибути', 'DMA': 'Доминика', 'DNK': 'Дания', 'DOM': 'Доминикана',
    'DZA': 'Алжир', 'ECU': 'Эквадор', 'EGY': 'Египет', 'ESP': 'Испания', 'EST': 'Эстония',
    'ETH': 'Эфиопия', 'FIN': 'Финляндия', 'FJI': 'Фиджи', 'FRA': 'Франция',
    'FRO': 'Фарерские острова', 'GAB': 'Габон', 'GBR': 'Великобритания',
    'GEO': 'Грузия', 'GGY': 'Гернси', 'GHA': 'Гана', 'GIB': 'Гибралтар',
    'GLP': 'Гваделупа', 'GNB': 'Гвинея-Бисау', 'GRC': 'Греция', 'GTM': 'Гватемала',
    'GUY': 'Гайана', 'HKG': 'Гонконг', 'HND': 'Гондурас', 'HRV': 'Хорватия',
    'HUN': 'Венгрия', 'IDN': 'Индонезия', 'IMN': 'Остров Мэн', 'IND': 'Индия',
    'IRL': 'Ирландия', 'IRN': 'Иран', 'IRQ': 'Ирак', 'ISL': 'Исландия', 'ISR': 'Израиль',
    'ITA': 'Италия', 'JAM': 'Ямайка', 'JEY': 'Джерси', 'JOR': 'Иордания',
    'JPN': 'Япония', 'KAZ': 'Казахстан', 'KEN': 'Кения', 'KHM': 'Камбоджа',
    'KIR': 'Кирибати', 'KNA': 'Сент-Китс и Невис', 'KOR': 'Южная Корея',
    'KWT': 'Кувейт', 'LAO': 'Лаос', 'LBN': 'Ливан', 'LBY': 'Ливия', 'LCA': 'Сент-Люсия',
    'LIE': 'Лихтенштейн', 'LKA': 'Шри-Ланка', 'LTU': 'Литва', 'LUX': 'Люксембург',
    'LVA': 'Латвия', 'MAC': 'Макао', 'MAR': 'Марокко', 'MCO': 'Монако',
    'MDG': 'Мадагаскар', 'MDV': 'Мальдивы', 'MEX': 'Мексика', 'MKD': 'Северная Македония',
    'MLI': 'Мали', 'MLT': 'Мальта', 'MMR': 'Мьянма', 'MNE': 'Черногория',
    'MOZ': 'Мозамбик', 'MRT': 'Мавритания', 'MUS': 'Маврикий', 'MWI': 'Малави',
    'MYS': 'Малайзия', 'MYT': 'Майотта', 'NAM': 'Намибия', 'NCL': 'Новая Каледония',
    'NGA': 'Нигерия', 'NIC': 'Никарагуа', 'NLD': 'Нидерланды', 'NOR': 'Норвегия',
    'NPL': 'Непал', 'NZL': 'Новая Зеландия', 'OMN': 'Оман', 'PAK': 'Пакистан',
    'PAN': 'Панама', 'PER': 'Перу', 'PHL': 'Филиппины', 'PLW': 'Палау', 'POL': 'Польша',
    'PRI': 'Пуэрто-Рико', 'PRT': 'Португалия', 'PRY': 'Парагвай', 'PYF': 'Французская Полинезия',
    'QAT': 'Катар', 'ROU': 'Румыния', 'RUS': 'Россия', 'RWA': 'Руанда', 'SAU': 'Саудовская Аравия',
    'SDN': 'Судан', 'SEN': 'Сенегал', 'SGP': 'Сингапур', 'SLE': 'Сьерра-Леоне',
    'SLV': 'Сальвадор', 'SMR': 'Сан-Марино', 'SRB': 'Сербия', 'STP': 'Сан-Томе и Принсипи',
    'SUR': 'Суринам', 'SVK': 'Словакия', 'SVN': 'Словения', 'SWE': 'Швеция',
    'SYC': 'Сейшелы', 'SYR': 'Сирия', 'TGO': 'Того', 'THA': 'Таиланд', 'TJK': 'Таджикистан',
    'TMP': 'Восточный Тимор', 'TUN': 'Тунис', 'TUR': 'Турция', 'TWN': 'Тайвань',
    'TZA': 'Танзания', 'UGA': 'Уганда', 'UKR': 'Украина', 'UMI': 'Внешние малые острова США',
    'URY': 'Уругвай', 'USA': 'США', 'UZB': 'Узбекистан', 'Unknown': 'Неизвестно',
    'VEN': 'Венесуэла', 'VGB': 'Британские Виргинские острова', 'VNM': 'Вьетнам',
    'ZAF': 'ЮАР', 'ZMB': 'Замбия', 'ZWE': 'Зимбабве'
}

# Шаг 2: Создаём DataFrame
countries_df = pd.DataFrame([
    {'country_code': code, 'country_name': name}
    for code, name in ru_code_name.items()
])

# Шаг 3: Загрузка
try:
    countries_df.to_sql('countries', engine, if_exists='append', index=False)
    print('Все страны из словаря загружены в таблицу countries.')
except IntegrityError as e:
    print('countries: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('countries: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('countries: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('countries: неизвестная ошибка при загрузке:', e)

Все страны из словаря загружены в таблицу countries.


#### 5.2.2. букинг

In [95]:
# Шаг 1: Создаём DataFrame
booking_df = df[[
    'arrival_date',
    'reservation_status',
    'reservation_status_date',
    'lead_time',
    'days_in_waiting_list',
    'stays_in_weekend_nights',
    'stays_in_week_nights',
    'assigned_room_type',
    'deposit_type',
    'hotel'
]].copy()

# Шаг 2: Загрузка
try:
    booking_df.to_sql('booking', engine, if_exists='append', index=False)
    print('booking: данные успешно загружены в таблицу')
except IntegrityError as e:
    print('booking: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('booking: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('booking: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('booking: неизвестная ошибка при загрузке:', e)

booking: данные успешно загружены в таблицу


#### 5.2.3. поведение

In [97]:
# Шаг 1: Получаем booking_id из базы
try:
    with engine.begin() as conn:
        booking_ids = pd.read_sql("SELECT booking_id FROM booking ORDER BY booking_id", conn)
    print('booking_id: данные успешно получены из базы')
except IntegrityError as e:
    print('booking_id: ошибка целостности данных при SELECT:', e)
except KeyError as e:
    print('booking_id: ошибка структуры при SELECT:', e)
except UnicodeError as e:
    print('booking_id: ошибка кодировки при SELECT:', e)
except Exception as e:
    print('booking_id: неизвестная ошибка при получении данных:', e)

# Шаг 2: Присваиваем обратно к df
try:
    df = df.reset_index(drop=True)
    df['booking_id'] = booking_ids['booking_id'].values
    print('booking_id: успешно присвоен обратно к df')
except IntegrityError as e:
    print('booking_id: ошибка целостности данных при присвоении:', e)
except KeyError as e:
    print('booking_id: ошибка структуры при присвоении:', e)
except UnicodeError as e:
    print('booking_id: ошибка кодировки:', e)
except ValueError as e:
    print('booking_id: несоответствие длины между df и booking_ids:', e)
except Exception as e:
    print('booking_id: неизвестная ошибка при присвоении:', e)

# Шаг 3: Собираем таблицу
client_behavior_df = df[[
    'booking_id',
    'booking_changes',
    'is_canceled',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled'
]].copy()

client_behavior_df['is_canceled'] = client_behavior_df['is_canceled'].astype(bool)
client_behavior_df['is_repeated_guest'] = client_behavior_df['is_repeated_guest'].astype(bool)

# Шаг 4: Загрузка
try:
    client_behavior_df.to_sql('client_behavior', engine, if_exists='append', index=False)
    print('client_behavior: данные успешно загружены в таблицу')
except IntegrityError as e:
    print('client_behavior: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('client_behavior: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('client_behavior: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('client_behavior: неизвестная ошибка при загрузке:', e)

booking_id: данные успешно получены из базы
client_behavior: данные успешно загружены в таблицу


#### 5.2.4. запросы

In [99]:
# Шаг 1: Собираем таблицу
client_request_df = df[[
    'booking_id',
    'total_of_special_requests',
    'required_car_parking_spaces',
    'reserved_room_type',
    'adults',
    'children',
    'babies',
    'meal'
]].copy()

# Шаг 2: Загрузка
try:
    client_request_df.to_sql('client_request', engine, if_exists='append', index=False)
    print('client_request: данные успешно загружены в таблицу')
except IntegrityError as e:
    print('client_request: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('client_request: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('client_request: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('client_request: неизвестная ошибка при загрузке:', e)

client_request: данные успешно загружены в таблицу


#### 5.2.5. маркетинг

In [101]:
# Шаг 1: Получаем справочник стран (code → id)
try:
    with engine.begin() as conn:
        countries_df = pd.read_sql("SELECT country_id, country_code FROM countries", conn)
    print('marketing_info: справочник стран успешно получен из базы')
except IntegrityError as e:
    print('marketing_info: ошибка целостности данных при SELECT из countries:', e)
except KeyError as e:
    print('marketing_info: ошибка структуры при SELECT из countries:', e)
except UnicodeError as e:
    print('marketing_info: ошибка кодировки при SELECT из countries:', e)
except Exception as e:
    print('marketing_info: неизвестная ошибка при получении справочника стран:', e)

# Шаг 2: Присоединяем country_id к df
df = df.merge(countries_df, how='left', left_on='country', right_on='country_code')

# Шаг 3: Собираем таблицу
marketing_info_df = df[[
    'booking_id',
    'country_id',
    'market_segment',
    'distribution_channel',
    'agent',
    'customer_type',
    'adr'
]].copy()

# Шаг 4: Загрузка
try:
    marketing_info_df.to_sql('marketing_info', engine, if_exists='append', index=False)
    print('marketing_info: данные успешно загружены в таблицу')
except IntegrityError as e:
    print('marketing_info: ошибка целостности данных (возможно дубликаты или нарушение связей):', e)
except KeyError as e:
    print('marketing_info: ошибка структуры DataFrame (отсутствует нужная колонка):', e)
except UnicodeError as e:
    print('marketing_info: ошибка кодировки текста (неподдерживаемые символы):', e)
except Exception as e:
    print('marketing_info: неизвестная ошибка при загрузке:', e)

marketing_info: справочник стран успешно получен из базы
marketing_info: данные успешно загружены в таблицу
